In [ ]:
import math

import cv2
from matplotlib import pyplot as plt

from functions.axes_distribution_functions import UniformDistributionAxesGenerator
from src.functions.center_points import GaussianAlgorithmCenterGenerator


In [ ]:
from src.functions.poisson_sampling import poisson_sampling

print("Test Poisson Sampling")
print("=" * 30)

width, height, r, k = 100, 100, 5, 30
points = poisson_sampling(width, height, r)

print(f"Wygenerowano {len(points)} punktów")
print(f"Obszar: {width} x {height}")
print(f"Minimalna odległość: {r}")

x_coords = [p[0] for p in points]
y_coords = [p[1] for p in points]

plt.figure(figsize=(8, 8))
plt.scatter(x_coords, y_coords, s=20, alpha=0.7)
plt.xlim(0, width)
plt.ylim(0, height)
plt.title(f'Poisson Sampling - {len(points)} punktów')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print(f"\nPierwsze 5 punktów:")
for i, (x, y) in enumerate(points[:5]):
    print(f"  {i+1}: ({x:.2f}, {y:.2f})")

In [ ]:
import noise
import numpy as np
from src.utils.classes.nuclei import NucleiOld


class TestNuclei(NucleiOld):
    def __init__(self, center, axes, angle=0, color=(160, 83, 179), thickness=-1, irregularity=0.3, border_color=(107, 26, 121), border_thickness=2):
        super().__init__(center, axes, angle, color, thickness)
        self.irregularity = irregularity
        self.border_color = border_color
        self.border_thickness = border_thickness
        self.seed = np.random.randint(0, 100)

    def draw_nuclei(self, image):
        cx, cy = self.center
        ax, ay = self.axes
        angle = np.deg2rad(self.angle)

        points = []
        num_points = 1000

        for i in range(num_points):
            t = 2 * math.pi * i / num_points
            x = ax * np.cos(t)
            y = ay * np.sin(t)

            perlin_value = noise.pnoise1(t * 2.0,
                                         octaves=4,
                                         persistence=0.5,
                                         lacunarity=3.0,
                                         repeat=1024,
                                         base=self.seed)
            factor = 1 + self.irregularity * perlin_value

            x *= factor
            y *= factor

            xr = x * np.cos(angle) - y * np.sin(angle)
            yr = x * np.sin(angle) + y * np.cos(angle)

            points.append([int(cx + xr), int(cy + yr)])

        points = np.array(points, dtype=np.int32).reshape((-1, 1, 2))
        cv2.fillPoly( image, [points], self.color )
        if self.border_thickness > 0:
            cv2.polylines(image, [points], isClosed=True, color=self.border_color, thickness=self.border_thickness)


In [ ]:
from src.functions.ellipse_params import ellipse_proportion

image = np.zeros((300, 300, 3),   dtype=np.uint8)

n3 = TestNuclei((130,130), ellipse_proportion(50),irregularity=0.1,thickness=2)

n3.draw_nuclei(image)

cv2.imshow('Nucleus', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from src.utils.classes.cancer_nucleus import CancerNucleusOld

image = np.zeros((300, 300, 3),   dtype=np.uint8)

n3 = CancerNucleusOld((130, 130), ellipse_proportion(50), irregularity=0.1, thickness=2)

n3.draw_nuclei(image)

cv2.imshow('Nucleus', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from src.data_generation.picture_generator import generate_picture_with_Poisson_sampling

image = generate_picture_with_Poisson_sampling(500, 500, proportionally=False)
cv2.imshow('generator', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from src.data_generation.single_cell_generator import single_cell_generator_with_open_window
from src.functions.ellipse_params import calculate_axes_size_from_image_size


proba = calculate_axes_size_from_image_size(500, 500)

cell = single_cell_generator_with_open_window(500, 500)
proba


In [ ]:
from src.functions.ellipse_params import calculate_axes_size_from_image_size

proba = calculate_axes_size_from_image_size(28, 38)
proba

In [ ]:
from src.utils.cell_settings import border_line_thickness

tryy = border_line_thickness(500, 500)
tryy

In [ ]:
from functools import partial
from src.data_generation.single_cell_generator import single_cell_generator_with_return_image
from src.data_generation.picture_generator import generate_picture_with_rows_and_columns

probaaa = partial(single_cell_generator_with_return_image,100,100)
prooooba = generate_picture_with_rows_and_columns(2, 3, probaaa)


In [ ]:
from src.data_generation.picture_generator import generate_image_with_random_aligment, show_image

proba_losowego = generate_image_with_random_aligment()

show_image(proba_losowego)

In [ ]:
from src.functions.ellipse_params import cell_size_proportionally

rozmiar_komorki = cell_size_proportionally(128,128)
rozmiar_komorki

In [ ]:
from src.functions.ellipse_params import cell_size_proportionally
print(cell_size_proportionally(128, 128))

In [1]:
from src.functions.axes_distribution_functions import NormalDistributionAxesGenerator
from src.functions.center_points import PoissonAlgorithmCenterGenerator, ClusteredAlgorithmCenterGenerator, GaussianAlgorithmCenterGenerator
from src.utils.classes.cancer_nucleus import CancerNucleus

center= PoissonAlgorithmCenterGenerator(128, 128,radius=30, k=20)
center.prepare_iterator()
axes = NormalDistributionAxesGenerator(10, 8, 7, 2)


center.generate_points()

Generated 15 centers


[[1, 57],
 [38, 10],
 [4, 18],
 [68, 19],
 [74, 51],
 [45, 70],
 [21, 96],
 [54, 103],
 [120, 49],
 [98, 13],
 [94, 77],
 [90, 117],
 [20, 127]]

In [3]:
from src.data_generation.picture_generator import create_slide_image

create_slide_image(center, axes)

In [4]:
from src.functions.axes_distribution_functions import UniformDistributionAxesGenerator

center1 = ClusteredAlgorithmCenterGenerator(128, 128, 10, 3)
center1.prepare_iterator()
center1.generate_points()
axes1 = UniformDistributionAxesGenerator(10, 8, 2)
create_slide_image(center1, axes1, )



Generated 10 centers


In [7]:
center2 = GaussianAlgorithmCenterGenerator(128,128,9,40)
center2.prepare_iterator()
create_slide_image(center2, axes)

Generated 9 centers


In [8]:
from src.functions.center_points import RandomAlignmentCenterGenerator

center3 = RandomAlignmentCenterGenerator(128,128,9,10)
center3.prepare_iterator()





Generated 9 centers


In [9]:
from src.utils.cell_settings import generate_color_variation_normal

generate_color_variation_normal((160, 83, 179))



(167, 81, 194)

In [12]:
from src.functions.center_points import ClusteredAlgorithmCenterGenerator


centra = NormalDistributionAxesGenerator(10, 8, 7, 2)
axes2 = ClusteredAlgorithmCenterGenerator(128, 128, 10, 3)
create_slide_image(axes2, centra)

Generated 10 centers
